# Práctica 2

## Ejercicio 1

Consulte el man de fork().
Escriba un proceso p1 que cree tres procesos hijo y espere en un bucle a recibir una señal utilizando la 
función pause(). Incluya un sleep en los procesos hijo para que finalicen unos segundos después de 
ser creados. El proceso padre deberá imprimir un mensaje en pantalla cada vez que finalice un proceso 
hijo, y finalizar cuando lo hayan hecho todos sus hijos.

### Fork(): 

>creates a new process by duplicating the calling process. The new process is referred to as the child process.  The calling process is referred to as the parent process. The child process and the parent process run in separate memory spaces.  At the time of fork() both memory spaces have the same content.  Memory writes, file mappings (mmap(2)), and unmappings (munmap(2)) performed by one of the processes do not affect the other.

[Ver el man](https://man7.org/linux/man-pages/man2/fork.2.html)

**Notas importantes:**
- The child does not inherit timers from its parent.
- On success, the PID of the child process is returned in the parent, and 0 is returned in the child.  On failure, -1 is returned in the parent, no child process is created, and errno is set to indicate the error.

In [7]:
//Ejemplo de fork()
#include <stdio.h>
#include <sys/types.h>
#include <unistd.h>

int main() {
  pid_t pid;
  pid = fork();
  if (pid == 0) {
    printf("Soy el hijo, mi ID es %d y mi padre es %d\n", getpid(), getppid());
  } else if (pid > 0) {
    printf("Soy el padre, mi ID es %d y el ID de mi hijo es %d\n", getpid(), pid);
  } else {
    printf("Error al crear el proceso hijo\n");
  }
  return 0;
}

Soy el padre, mi ID es 7969 y el ID de mi hijo es 7972
Soy el hijo, mi ID es 7972 y mi padre es 7969


**SIGCHLD 17** es una señal enviada a un proceso padre cuando uno de sus procesos hijos termina o se detiene. La señal es generada por el sistema operativo y se utiliza para informar al proceso padre de que debe limpiar los recursos asociados con el proceso hijo terminado. Esto puede incluir la liberación de la memoria, el cierre de descriptores de archivos abiertos y otras tareas relacionadas. El proceso padre normalmente recibe esta señal a través de la llamada al sistema wait o waitpid, que bloquea al proceso padre hasta que un proceso hijo termine. El proceso padre luego puede recuperar el estado de salida del proceso hijo terminado y tomar cualquier acción necesaria.

In [1]:
#include <stdio.h>
#include <unistd.h>
#include <sys/wait.h>

int main(void) {
  pid_t pid;

  pid = fork();
  if (pid == 0) {
    printf("Soy el proceso hijo con PID %d\n", getpid());
    return 0;
  } else {
    int status;
    pid_t child_pid;

    child_pid = wait(&status);
    printf("Soy el proceso padre con PID %d\n", getpid());
    printf("El proceso hijo con PID %d ha terminado\n", child_pid);
    printf("El estado de salida del proceso hijo es %d\n", WEXITSTATUS(status));
  }
  
  return 0;
}

Soy el proceso hijo con PID 18085
Soy el proceso padre con PID 18082
El proceso hijo con PID 18085 ha terminado
El estado de salida del proceso hijo es 0


En este ejemplo, el proceso padre crea un proceso hijo mediante la llamada a fork(). El hijo imprime su PID y termina. El padre, a su vez, espera hasta que el hijo termine con la llamada a wait(), que recibe el estado de salida del hijo y su PID. Finalmente, el padre imprime información sobre el hijo y su estado de salida.

## Ejercicio 2
Consulte el man de wait() y exit(). Repita el Ejercicio 1 utilizando la función wait() en vez de pause() y sin capturar la señal SIGCHLD. Incluya la función exit() para finalizar cada proceso hijo. Cada vez que finalice un proceso hijo, el proceso padre deberá imprimir en pantalla el pid del proceso finalizado y el código de finalización utilizado en el exit() por dicho proceso hijo.



### wait()
 >All of these system calls are used to wait for state changes in a child of the calling process, and obtain information about the child whose state has changed.  A state change is considered to be: the child terminated; the child was stopped by a signal; or the child was resumed by a signal.  In the case of a terminated child, performing a wait allows the system to release the resources associated with the child; if a wait is not performed, then the terminated child remains in a "zombie" state (see NOTES below).

[ Verl el man](https://man7.org/linux/man-pages/man2/wait.2.html)

**Notas importantes:**
- Nota

### exit()
 >The exit() function causes normal process termination and the least significant byte of status (i.e., status & 0xFF) is returned to the parent (see wait(2)).

[Ver el man](https://man7.org/linux/man-pages/man3/exit.3.html)

**Notas importantes:**
- Nota

In [3]:
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>

int main()
{
    int pid = fork();

    if (pid == 0) {
        printf("Soy el hijo con PID %d\n", getpid());
        exit(23);
    }
    else {
        int status;
        wait(&status);
        printf("Soy el padre con PID %d\n", getpid());
        printf("El hijo con PID %d ha terminado con código %d\n", pid, WEXITSTATUS(status));
    }

    return 0;
}


Soy el hijo con PID 7708
Soy el padre con PID 7705
El hijo con PID 7708 ha terminado con código 23


### WEXITSTATUS(wstatus)
>*returns the exit status of the child.  This consists of the least significant 8 bits of the status argument that the child specified in a call to exit(3) or _exit(2) or as the argument for a return statement in main().  This macro should be employed only if WIFEXITED returned true.

[Ver el man](https://man7.org/linux/man-pages/man2/wait.2.html#:~:text=returning%20from%20main().-,WEXITSTATUS(wstatus),-returns%20the%20exit)

WEXITSTATUS es una macro que se usa en el manejo de procesos en sistemas operativos Unix y Linux. Esta macro se utiliza para extraer el código de salida de un proceso que ha finalizado.

El código de salida de un proceso es un entero que indica el resultado de la ejecución de un proceso. Normalmente, un código de salida de 0 significa que el proceso se ha ejecutado con éxito, mientras que un código de salida distinto de cero indica que el proceso ha fallado.

Para obtener el código de salida de un proceso, primero debes esperar a que el proceso finalice usando la función wait o waitpid. Luego, puedes usar la macro WEXITSTATUS para extraer el código de salida del proceso. La macro WEXITSTATUS toma una variable de tipo int que contiene el resultado de la función wait o waitpid, y extrae el código de salida del proceso.

## Ejercicio 3
Consulte el man de waitpid() y del comando ps. Repita el Ejercicio 2 utilizando la función waitpid()1 en vez de wait(). Compruebe, utilizando el comando ps, cómo aparecen procesos zombie que no aparecían en el Ejercicio 2. Razone acerca de las ventajas/inconvenientes de utilizar waitpid() en vez de wait()
### waitpid()
> La  función  waitpid  suspende  la  ejecución  del  proceso  en  curso  hasta  que un hijo especificado por el argumento pid ha terminado, o hasta que  se  produce  una  señal  cuya acción es finalizar el proceso actual o llamar a la función manejadora de la señal.

>Si  el  hijo  especificado  por  pid ha terminado cuando se produce la llamada (un proceso "zombie"), la función vuelve inmediatamente. Todos los recursos del sistema reservados por el hijo son liberados.

[Ver el man](https://man7.org/linux/man-pages/man2/wait.2.html#:~:text=termed%20waitable.-,wait()%20and%20waitpid(),-The%20wait()%20system)

In [2]:
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <sys/wait.h>

int main()
{
    int pid = fork();

    if (pid == 0) {
        printf("Soy el hijo con PID %d\n", getpid());
        exit(22);
    }
    else {
        int status;
        waitpid(pid, &status, 0);
        printf("Soy el padre con PID %d\n", getpid());
        printf("El hijo con PID %d ha terminado con código %d\n", pid, WEXITSTATUS(status));
    }

    return 0;
}


Soy el hijo con PID 29536
Soy el padre con PID 29533
El hijo con PID 29536 ha terminado con código 22


## Ejercicio 4
Consulte el man de execl(). Repita el Ejercicio 2 escribiendo un nuevo programa p2 para los procesos hijo que imprima en pantalla  la lista de parámetros que reciba en la línea de comandos. Utilice execl() para invocar a dicho programa desde el código de los procesos hijo.

### execl()
>The exec family of functions replaces the current process image with a new process image.  The functions described in this manual page are front-ends for the function execve(2).  (See the manual page for execve(2) for detailed information about the replacement of the current process.)

>The initial argument for these functions is the pathname of a file which is to be executed. 

>The const char *arg0 and subsequent ellipses in the execl(), execlp(), and execle() functions can be thought of as arg0, arg1, ..., argn. Together they describe a list of one or more pointers to null-terminated strings that represent the argument list available to the executed program.  The first argument, by convention, should point to the file name associated with the file being executed.  The list of arguments must be terminated by a NULL pointer.

[Ver el man](http://www.manpagez.com/man/3/execl/)

**execl()** es una función que se utiliza en el sistema operativo Unix para reemplazar el programa actual con un nuevo programa. La función toma una serie de argumentos, donde el primer argumento es el nombre del programa que se va a ejecutar, y los siguientes argumentos son los argumentos que se le pasarán al programa.


In [5]:
#include <stdio.h>
#include <unistd.h>

int main()
{
    printf("Antes de execl()\n");
    execl("/bin/date", "date", NULL);
    printf("Después de execl()\n");
    return 0;
}


Antes de execl()
vie 17 feb 2023 08:08:00 UTC


En este ejemplo, el programa inicial se reemplaza con el comando **date**. Después de llamar a execl(), el programa original ya no se ejecuta, y en su lugar, el comando **date** se ejecuta y produce la salida que se espera.

Es importante notar que la función execl() no devuelve un valor, ya que el programa actual se reemplaza completamente con el nuevo programa, por lo que cualquier código después de la llamada a execl() no se ejecutará.

## Explicación de mi programa 5

- El proceso padre imprime "De Gea" y crea tres procesos hijos.
- El proceso hijo 1 imprime "Ramos" y termina.
- El proceso hijo 2 imprime "Piqué" y termina.
- El proceso hijo 3 imprime "Jordi Alba" y termina.
- El proceso padre imprime "Carvajal" y espera a que tres procesos hijos terminen.
- El proceso padre espera a que el proceso hijo 3 (Jordi Alba) termine, crea un nuevo proceso hijo (4) y espera a que tres procesos hijos terminen.
- El proceso hijo 4 imprime "Thiago" y "Silva" y termina.
- El proceso padre imprime "Busquets", "Isco" y "Aspas".
- Si Thiago Silva no ha terminado, el proceso padre espera a que termine.
- El proceso padre imprime "Morata".
